In [1]:
from SBM_SDE_tensor import *
from obs_and_flow import *
from mean_field import *
from training import *
import torch
#import seaborn as sns

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 200 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
pretrain_lr = 1e-3
elbo_lr = 1e-3
niter = 1000
piter = 2
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.
prior_scale_factor = 1 / 3 #Prior standard deviations set at 0.25 of prior means.
obs_error_scale_factor = 1 / 3

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [5]:
net1, q_theta1, ELBO_losses1, norm_losses1 = train(devi, pretrain_lr, elbo_lr, niter, piter, batch_size, prior_scale_factor, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_prior_means, temp_gen, temp_ref, analytical_steady_state_init_CON)



Train Diffusion:   0%|          | 0/1000 [00:00<?, ?it/s]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.9160,  1.7033,  0.4902],
         [ 2.3470,  0.9914,  1.4125],
         ...,
         [ 0.6135,  0.6657,  1.1950],
         [ 1.2483,  0.6506,  0.6374],
         [ 0.9606,  0.5711,  0.7368]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.9160,  0.9812,  0.8700],
         [ 1.3645,  1.6761,  0.6522],
         ...,
         [ 0.9946,  0.8902,  0.4969],
         [ 0.7666,  0.7376,  0.4077],
         [ 1.2567,  0.6628,  0.5896]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[1.2140, 0.8484, 0.7219],
        [1.2043, 0.8591, 0.7171]], grad_fn=<MeanBackward1>)

Moving average norm loss at <built-in function iter> iterations is: 1988.271240234375. Best norm loss value is: 1988.271240234375.

C_PATH mean = tensor([[1.2140, 0.8484, 0.7219],
        [1.2043, 0.8591, 0.7171]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],



Train Diffusion:   0%|          | 1/1000 [00:00<14:27,  1.15it/s]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 1.9523,  0.5985,  0.6514],
         [ 1.3369,  0.4985,  0.5567],
         ...,
         [ 1.4690,  0.5797,  0.5534],
         [ 0.9031,  0.5911,  0.8186],
         [ 1.1779,  0.5487,  0.8552]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.7760,  0.8911,  0.6148],
         [ 3.2105,  1.4573,  0.7428],
         ...,
         [ 0.7933,  0.6585,  0.4420],
         [ 0.8911,  0.5933,  0.4944],
         [ 0.6947,  0.7324,  1.0352]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[1.9397, 0.8428, 0.6005],
        [1.8209, 0.8694, 0.6057]], grad_fn=<MeanBackward1>)




Train Diffusion:   0%|          | 2/1000 [00:01<14:45,  1.13it/s]

...


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [3.8318e-01, 3.0343e-01, 2.1912e-01],
         [2.5215e-02, 2.5819e-01, 2.2350e-01],
         ...,
         [2.3327e-01, 5.3976e-02, 7.2338e-02],
         [2.2674e-01, 1.3729e-01, 6.9344e-02],
         [2.0465e-01, 5.0207e-02, 8.4254e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [1.5753e-01, 1.5486e-01, 3.6186e-02],
         [5.6080e-02, 1.7635e-01, 5.7694e-02],
         ...,
         [2.2452e-01, 2.1346e-01, 5.6539e-04],
         [2.2656e-01, 7.3250e-02, 2.3522e-03],
         [2.3685e-01, 1.5987e-01, 2.2648e-01]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[0.1369, 0.2701, 0.0523],
        [0.1371, 0.2675, 0.0539]], grad_fn=<MeanBackward1>)

Drift =  tensor([[[-1.1642e-10, -5.8208e-11,  0.0000e+00],
         [ 1.5182e-03, -2.0740e-03,  5.6127e-04],
         [ 1.4682e-03, -1.8044e-03,  4.6803e-04],
         ...,
      



Train Diffusion:  32%|███▏      | 321/1000 [04:54<09:29,  1.19it/s]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [1.5718e-01, 2.8521e-01, 3.2071e-01],
         [5.5747e-02, 1.5331e-01, 6.4994e-02],
         ...,
         [2.7557e-01, 5.8724e-02, 7.5505e-04],
         [2.0097e-01, 5.6581e-02, 5.6159e-02],
         [1.7842e-01, 5.8652e-02, 1.9749e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [3.8226e-01, 1.3072e-01, 4.1387e-02],
         [2.5064e-02, 2.5235e-01, 1.6223e-01],
         ...,
         [1.7957e-01, 1.2678e-01, 9.1174e-02],
         [2.2820e-01, 1.9549e-01, 2.1634e-03],
         [1.8208e-01, 1.7796e-01, 1.1199e-02]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[0.1366, 0.2679, 0.0525],
        [0.1369, 0.2673, 0.0510]], grad_fn=<MeanBackward1>)

Drift =  tensor([[[-1.1642e-10, -5.8208e-11,  0.0000e+00],
         [ 1.4964e-03, -1.9413e-03,  5.0370e-04],
         [ 1.2745e-03, -1.0329e-03,  2.9257e-04],
         ...,
      



Train Diffusion:  32%|███▏      | 322/1000 [04:56<10:23,  1.09it/s]


KeyboardInterrupt: 

In [ ]:
net2, q_theta2, ELBO_losses2, norm_losses2 = train(devi, pretrain_lr, elbo_lr, niter, piter, batch_size, prior_scale_factor, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_prior_means, temp_gen, temp_ref, analytical_steady_state_init_CON, LEARN_THETA = True)